# Install External Dependencies

In [ ]:
pip install -q tensorflow pydrive kaggle

In [ ]:
from google.colab import files

files.upload()

# Loading Kaggle API for Fetching Dataset

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/datasets/ocular-disease-recognition-odir5k.zip" -d odir5k

# Import External Dependencies

In [17]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import pandas as pd
import numpy as np
import os
import itertools
import io
import tensorflow as tf

from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# visualization dependencies
import cv2
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

from plotly.offline import iplot, plot
from plotly.offline import init_notebook_mode

import cufflinks as cf
import warnings

warnings.filterwarnings('ignore')

cf.go_offline()
cf.set_config_file(offline=False, world_readable=False)

init_notebook_mode(connected=False)

plt.rcParams['figure.dpi'] = 120.0
plt.rcParams['font.sans-serif'] = 'Lato'
sns.set(style='whitegrid', palette='muted',
              rc={'figure.figsize': (16, 16)})

tf.config.run_functions_eagerly(True)



In [18]:
IMG_SIZE = 256
NUM_EPOCHS = 50
LR = 1e-4
BATCH_SIZE = 32
NUM_CLASSES = 4

In [19]:
import shutil

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
    <script src="/static/components/requirejs/require.js"></script>
    <script>
      requirejs.config({
        path: {
          base: '/static/base',
          plotly: "https://cdn.plot.ly/plotly-1.5.1.min.js?noext",
        },
      });
    </script>
  '''))

def clear_content(directory):
  for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    try:
      if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)
      elif os.path.isdir(file_path):
        shutil.rmtree(file_path)
    except Exception as exc:
      raise("Failed to Delete: %s %s" % (file_path, exc))

# Step 1: Load and Inspect the Dataset

In [20]:
!conda install -y gdown
!gdown --id 1DDQiPKVscCJggU9EHd334mqNqpC6ZWvQ

In [21]:
!gdown --id 1-749mKKw7du1verM4MNsTKXQcGkICEQf
!gdown --id 1mTG7Sj9ouZ49lLJFZTtEMM4Ryal4PIQ6

In [22]:
!gdown --id 1XLClixZesnrPoF_wANWWEPtZZeR1_ZtH

In [23]:
!gdown --id 1-3Jr_Oi2uaHnCJpZmEBWDj3R1uz2W7rn
!gdown --id 1xwZr_LLSh1TeUYzSHaekwYYU6FYcNVWw

In [24]:
!gdown --id 1TVtiqUwaEVmi3GQ4RHrtOuSb7Sfxlqop
!gdown --id 1RNuKChSrkmXWyEqqiL34fj-Y3evXheOp # rgb directory
!gdown --id 1EIiDPTsmIEzHwnRid9AteteUKwu_HPem
!gdown --id 1DMsCTSv16WioN2d20__UGhmExEUszKza # rgb_merged_model

In [25]:
!pip install openpyxl

In [75]:
from zipfile import ZipFile
from tqdm import tqdm

prep_path = "/kaggle/working/preprocessed_images.zip"
odir_path = "/kaggle/working/ocular-disease-recognition-odir5k.zip"

paths = ['odir_rgb_merged_model_four-0.9460.zip','odir_rgb_model_four-0.9549.zip','ocular-disease-recognition-odir5k.zip', 'preprocessed_images.zip', 'odir_model_four-0.89.zip','odir_model_four-0.9407.zip', 'RGB_odir_model_four-0.9162.zip', 'odir_model_eight-0.9059.zip']
names = ['odir_rgb_merged_model_four-0.9460','odir_rgb_model_four-0.9549','odir5k', 'preprocessed_images', 'odir_model_four-0.89', 'odir_model_four-0.9407','RGB_odir_model_four-0.9162','odir_model_eight-0.9059']

for i in range(6):
    if os.path.exists(names[i]):
        clear_content(names[i])
    else:
        os.mkdir(names[i])
        
    with ZipFile(paths[i], 'r') as obj:
        file_list = obj.namelist()
        for file in tqdm(file_list):
            obj.extract(file, names[i])
    print()
    print(f"{names[i]} Data extracted!")

##### Pre-Existing Models"

In [27]:
!gdown --id 1RDJ12CEHyX6lhgc4aCtqQFPUWdmTkZ0Q # DenseNet169
!gdown --id 1Q6p87UKnFffvyKIiLtltMy-cAdYVFTUK # InceptionV3
!gdown --id 1rH6zoiyUY8Cj9QxxeJe1UHI5xZLqUb-- # MobileNetv2
!gdown --id 10eGffNQo7G7kCTlFeEbutYkYTgBNaf_E # ResNet50V2

In [28]:
models_dir = "fine_tunedModels"

if os.path.exists(models_dir):
    clear_content(models_dir)
else:
    os.mkdir(models_dir)

paths = ['odir_rgb_model_four_densenet169-0.9188.zip', 'odir_rgb_model_four_inceptionv3-0.8531.zip', 
         'odir_rgb_model_four_mobilenetv2-0.8698.zip','odir_rgb_model_four_resnbet50v2-0.7526.zip']
names = [os.path.join(models_dir, 'densenet169'), os.path.join(models_dir,'inceptionv3'), 
         os.path.join(models_dir, 'mobilenetv2'), os.path.join(models_dir, 'resnet50v2')]

for i in range(4):
    if os.path.exists(names[i]):
        clear_content(names[i])
    else:
        os.mkdir(names[i])
        
    with ZipFile(paths[i], 'r') as obj:
        file_list = obj.namelist()
        for file in tqdm(file_list):
            obj.extract(file, names[i])
    print()
    print(f"{names[i]} Data extracted!")

In [29]:
for dirname, subfolders, filenames in os.walk("odir5k"):
  for subf in subfolders:
    print(os.path.join(dirname, subf))

In [30]:
for dirname, subfolders, filenames in os.walk("odir_rgb_merged_model_four-0.9460"):
  for file in filenames:
    print(os.path.join(dirname, file))

In [31]:
train_dir = "odir5k/ODIR-5K/ODIR-5K/Training Images"
test_dir = "odir5k/ODIR-5K/ODIR-5K/Testing Images"
annotations_path = "odir5k/full_df.csv"
prep_train = "preprocessed_images/preprocess_train"
prep_test = "preprocessed_images/preprocess_test"
discarded_path = "DiscardedImages.xlsx"
# odir_model_dir = "odir_model_eight-0.9059/trained_models/odir"
# odir_model_dir = "odir_model_eight-0.9059/trained_models/odir/odir_model-0.91"
# odir_dir = "odir_model_eight-0.9059/trained_models/odir"
odir_model_merged = "odir_rgb_merged_model_four-0.9460/trained_models/odir/odir_model-0.95"
odir_history_merged = "odir_rgb_merged_model_four-0.9460/trained_models/odir/odir_history.h5"
odir_model_path = "odir_model_four-0.9407/trained_models/odir/odir_model-0.94"
odir_rgb_model_path = "RGB_odir_model_four-0.9162/trained_models/odir/odir_model-0.92"
files_train = sorted(os.listdir(train_dir))
files_test = sorted(os.listdir(test_dir))

print("train: " , len(files_train))
print("test: ", len(files_test))

In [32]:
len(os.listdir(prep_train))

#### Data Loading

In [33]:
# pandas filtering functions

def data_mapping(df, initial):
  labels = []
  fundus_names = list(df.iloc[:, 3])

  for i in range(len(fundus_names)):
    labels.append(initial)

  return labels

def count_labels(df):
  labels = list(df.iloc[:, -12:-4].sum().index)
#   labels = ['N', 'C']
  total_count = []
  labels_count = []

  for i in range(len(labels)):
    label_count = data_mapping(df.loc[(df[labels[i]] == 1)], labels[i])
    total_count += label_count
    labels_count.append(len(label_count))
    print(f"{labels[i]}: {len(label_count)}")

  return total_count, labels_count

In [34]:
annot_df = pd.read_csv(annotations_path)
annot_df.info()

In [35]:
annot_df.iloc[:, -12:-4].sum().index

In [36]:
annot_df.head(10)

In [37]:
discarded_sheet = pd.read_excel(discarded_path, sheet_name="Sheet1")
# annot_df.loc[(annot_df["Left-Fundus"].isin(discarded_sheet["Left-Fundus"]))].drop(discarded_sheet['Left-Fundus'], axis=0)
dropped = annot_df.loc[~annot_df['Left-Fundus'].isin(discarded_sheet['Left-Fundus'])]
print(len(dropped.loc[(dropped['Left-Fundus'].isin(discarded_sheet['Left-Fundus']))]))
print(len(dropped.loc[(dropped['Left-Fundus'].str.match("4601_left.jpg"))]))

# ---- these below are not working ---- #
# duplicates = pd.merge(annot_df['Left-Fundus'], discarded_sheet['Left-Fundus'],
#                       how='inner', left_on=['Left-Fundus'], right_on=['Left-Fundus'],
#                       left_index=True)
# dropped = annot_df.loc[(annot_df['Left-Fundus'].isin(discarded_sheet['Left-Fundus']))].drop("Left-Fundus", axis=0)
# dropped = annot_df.drop(duplicates.index, axis=0)
# dropped = annot_df['Left-Fundus'].drop(discarded_sheet['Left-Fundus'])
# dropped.loc[(dropped['Left-Fundus'].isin(discarded_sheet['Left-Fundus']))]
# left_eyes = list(discarded_sheet.iloc[:, 1])
# len(left_eyes)
# annot_df.loc[(annot_df['Left-Diagnostic Keywords'].str.contains("|".join(["lens dust", "low image", "anterior", "optic disk", "refractive"])))]

In [38]:
# drop occluded and unclear images
discarded_sheet = pd.read_excel(discarded_path, sheet_name="Sheet1")
disc_tokens = ["low image", "lens dust", "anterior", "refractive","invisible","image", "image offset","chorioretinopathy"]

annot_df_ = annot_df.loc[~annot_df['Left-Fundus'].isin(discarded_sheet['Left-Fundus'])]
annot_df = annot_df.loc[~annot_df['Right-Fundus'].isin(discarded_sheet['Right-Fundus'])]
annot_df = annot_df.loc[(~annot_df['Left-Diagnostic Keywords'].str.
                         contains("|".join(disc_tokens)))]
annot_df = annot_df.loc[(~annot_df['Right-Diagnostic Keywords'].str.
                         contains("|".join(disc_tokens)))]
# test if such exist
print(len(annot_df.loc[(annot_df['Left-Diagnostic Keywords'].str.
                        contains("|".join(disc_tokens)))]))
len(annot_df)

In [39]:
total_count, labels_count = count_labels(annot_df)
len(total_count)

#### Load and Split Data

In [40]:
# drop non-existent images
non = []
for location in tqdm(annot_df.iloc[:]['Left-Fundus']):
    if not os.path.exists(os.path.join(prep_train, location)):
        non.append(location)
        
for location in tqdm(annot_df.iloc[:]['Right-Fundus']):
    if not os.path.exists(os.path.join(prep_train, location)):
        non.append(location)

annot_df = annot_df.loc[(~annot_df['Left-Fundus'].str.match('|'.join(non)))]
annot_df = annot_df.loc[(~annot_df['Right-Fundus'].str.match('|'.join(non)))]
len(annot_df)

## Step 2: Data Preprocessing

**Method 3: Crop Only and Resize Image**
* Leaves image as RGB

In [ ]:
def crop_image(image, plot=False, image_size=(256, 256)):
  # mask of colored pixels
  mask = image > 0

  # coordinates of colored pixels
  coordinates = np.argwhere(mask)

  # binding box of non-black pixels
  x0, y0, s0 = coordinates.min(axis=0)
  x1, y1, s1 = coordinates.max(axis=0) + 1 # slices are exclusive at the top

  # get the contents of the bounding box
  cropped = image[x0:x1, y0:y1]

  # convert to YUV for equalization
  img_yuv = cv2.cvtColor(cropped, cv2.COLOR_RGB2YUV)

  # apply adaptive equalization
  clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
  img_yuv[:, :, 0] = clahe.apply(img_yuv[:, :, 0])

  # apply image normalization
  mask = np.zeros((256, 256))
  normalized = cv2.normalize(img_yuv[:,:,0], mask, 0, 255, cv2.NORM_MINMAX)

  # convert back to rgb
  new_image = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

  new_image = cv2.resize(new_image, image_size, interpolation=cv2.INTER_AREA)

  if plot:
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.title(f"Original - {image.shape}")
    plt.axis('off')
    plt.grid(False)

    # ----------------- #

    plt.subplot(1,2,2)
    plt.imshow(new_image)
    plt.title(f"Preprocessed - {new_image.shape}")
    plt.axis('off')
    plt.grid(False)
    plt.show()
  
  return cropped

In [ ]:
import traceback

try:
  rand_idx = np.random.randint(0, len(files_train))
  image = cv2.imread(os.path.join(train_dir, files_train[rand_idx]))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  new_image = crop_image(image, plot=True, image_size=(256, 256))
except Exception as exc:
  traceback.print_tb(exc.__traceback__)

# Step 3: Data Mapping
* Mapping retinal images to their respective classes (disease)
* Start with two classes (Cataract and Normal)

In [41]:
# upsample function
def upsample_columns(df, max_size):
    df = df.sample(frac=1).reset_index(drop=True)
    perc = len(df) / max_size
    
    size = int(max_size - (perc * max_size))
    print("upsampled by: ", size)
    for i in range(size):
        rand_idx = np.random.randint(0, len(df))
#         eye = df[rand_idx]
        eye = df.at[rand_idx, 'Left-Fundus']
        row = df.loc[df['Left-Fundus'].str.match(eye)]
        
        if len(row) > 1:
            df = df.append(row.iloc[0], ignore_index=True)
        else:
            df = df.append(row, ignore_index=True)
#         df = pd.concat([df, row]).reset_index(drop=True)
#     df = pd.Series(df)
    return df

In [42]:
# eye = df_hypertension.at[322, 'Left-Fundus']
# row = df_hypertension.loc[df_hypertension['Left-Fundus'].str.match(eye)]
# # conc = pd.concat([df_hypertension, row])
# conc = df_hypertension.append(row.iloc[0], ignore_index=True)
# # row.iloc[0]
# conc

In [43]:
# print(df_slim.at[432, 'Left-Fundus'])

In [44]:
# slim to necessary columns
df_slim = annot_df.iloc[:, 1:15]
df_slim.head()

### Extract Cataract Images (Left and Right Retinas)

In [45]:
# extract left cataract images
df_left_cat = df_slim[df_slim['Left-Diagnostic Keywords'].str.match('cataract')]
df_left_cat.head()

In [46]:
# extract right cataract images
df_right_cat = df_slim[df_slim['Right-Diagnostic Keywords'].str.match('cataract')]
df_right_cat.head()

In [47]:
annot_df.loc[annot_df['D'] == 1, 'Left-Fundus']

### Combine Cataract Left and Right Images

In [48]:
# df_cat_filenames = df_left_cat['Left-Fundus'].append(df_right_cat['Right-Fundus'], ignore_index=True)
df_cat_filenames = pd.concat([df_left_cat, df_right_cat])
df_cat_filenames.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_cat_filenames.head()

In [49]:
df_cat_filenames.tail()

In [50]:
len(df_cat_filenames)

### Extract Normal Retinal Images (Left and Right)

In [51]:
# extract left norms
df_left_norm = df_slim.loc[df_slim['Left-Diagnostic Keywords'].str.match('normal')]
df_left_norm.head()

In [52]:
# extract right norm
df_right_norm = df_slim.loc[df_slim['Right-Diagnostic Keywords'].str.match('normal')]
df_right_norm.head()

### Combine Normal Images (Left and Right)

In [53]:
# df_norm_filenames = df_left_norm['Left-Fundus'].append(df_right_norm['Right-Fundus'], ignore_index=True)
df_norm_filenames = pd.concat([df_left_norm, df_right_norm])
df_norm_filenames.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_norm_filenames.head()

In [54]:
df_norm_filenames.tail()

In [55]:
len(df_norm_filenames)

In [56]:
# extract diabetes
df_left_diabetes = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('|'.join(['diabetic', 'proliferative'])))]
df_right_diabetes = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('|'.join(['diabetic', 'proliferative'])))]

# combine diabetes
# df_diabetes = df_left_diabetes['Left-Fundus'].append(df_right_diabetes['Right-Fundus'], ignore_index=True)
df_diabetes = pd.concat([df_left_diabetes, df_right_diabetes])
df_diabetes.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
print("diabetes: ", len(df_diabetes))

In [57]:
# extract glaucoma
df_left_glaucoma = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('glaucoma'))]
df_right_glaucoma = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('glaucoma'))]

# combine glaucoma
# df_glaucoma = df_left_glaucoma.append(df_right_glaucoma, ignore_index=True)
df_glaucoma = pd.concat([df_left_glaucoma, df_right_glaucoma])
df_glaucoma.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_glaucoma = upsample_columns(df_glaucoma, len(df_cat_filenames))
print("glaucoma: ", len(df_glaucoma))

In [58]:
# extract amd
df_left_amd = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('age-related'))]
df_right_amd = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('age-related'))]

# combine amd
# df_amd = df_left_amd['Left-Fundus'].append(df_right_amd['Right-Fundus'], ignore_index=True)
df_amd = pd.concat([df_left_amd, df_right_amd])
df_amd.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'],axis='columns', inplace=True)
df_amd = upsample_columns(df_amd, len(df_cat_filenames))
print("amd: ", len(df_amd))

In [59]:
# extract hypertension
df_left_hypertension = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('hypertensive'))]
df_right_hypertension = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('hypertensive'))]

# combine hypertension 
# df_hypertension = df_left_hypertension['Left-Fundus'].append(df_right_hypertension['Right-Fundus'], ignore_index=True)
df_hypertension = pd.concat([df_left_hypertension,df_right_hypertension]).reset_index(drop=True)
df_hypertension.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_hypertension = upsample_columns(df_hypertension, len(df_cat_filenames))
print("hypertension: ", len(df_hypertension))

In [60]:
# extract myopia
df_left_myopia = df_slim.loc[(df_slim['Left-Diagnostic Keywords'].str.contains('pathological myopia'))]
df_right_myopia = df_slim.loc[(df_slim['Right-Diagnostic Keywords'].str.contains('pathological myopia'))]

# combine myopia
# df_myopia = df_left_myopia['Left-Fundus'].append(df_right_myopia['Right-Fundus'], ignore_index=True)
df_myopia = pd.concat([df_left_myopia, df_right_myopia])
df_myopia.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_myopia = upsample_columns(df_myopia,len(df_cat_filenames))
print("myopia: ", len(df_myopia))

In [61]:
common_diseases = ['myopia', 'normal fundus', 'cataract', 'glaucoma', \
                   'hypertensive', 'proliferative', 'age-related', 'diabetic']
df_left_others = df_slim.loc[(~df_slim['Left-Diagnostic Keywords'].str.contains('|'.join(common_diseases)))]
df_right_others = df_slim.loc[(~df_slim['Right-Diagnostic Keywords'].str.contains('|'.join(common_diseases)))]

# combine others
# df_others = df_left_others['Left-Fundus'].append(df_right_others['Right-Fundus'], ignore_index=True)
df_others = pd.concat([df_left_others, df_right_others])
df_others.drop(['Patient Age', 'Patient Sex', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'], axis='columns', inplace=True)
df_others = upsample_columns(df_others, len(df_cat_filenames))
print("others: ", len(df_others))

### Select and Create a Random Sample
* we need to minimize the normal data to match the number of cataract samples

In [62]:
# There are 572 cataract images
df_norm_filenames_random = df_norm_filenames.sample(n=485)
df_norm_filenames_random.head()

In [63]:
df_norm_filenames_random = df_norm_filenames_random.reset_index(drop=True)
df_norm_filenames_random

In [64]:
# # display normal images
# plt.rcParams['figure.figsize'] = (10.0, 10.0)
# plt.subplots_adjust(wspace=0, hspace=0.2)

# for i in range(9):
#   rand_idx = np.random.randint(0, len(df_norm_filenames_random))
#   image = cv2.imread(os.path.join(train_output, df_norm_filenames_random[rand_idx]))
#   image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#   plt.subplot(3, 3, i+1)
#   plt.imshow(image, cmap='gray', vmin=0, vmax=255)
#   plt.title(f"{df_norm_filenames_random[rand_idx].split('.')[0]}: Normal")
#   plt.axis('off')
#   plt.grid(False)

# plt.show()

In [65]:
# df_normal_dn = df_normal.sample(n=381).reset_index(drop=True)
df_diabetes_dn = df_diabetes.sample(n=485).reset_index(drop=True)
df_others_dn = df_others.sample(n=485).reset_index(drop=True)
df_myopia_dn = df_myopia.sample(n=485).reset_index(drop=True)
df_amd_dn = df_amd.sample(n=485).reset_index(drop=True)
df_glaucoma_dn = df_glaucoma.sample(n=485).reset_index(drop=True)
# df_cataract_dn = df_cataract.sample(n=381).reset_index(drop=True)


In [66]:
# # df_normal_dn = df_normal.sample(n=381).reset_index(drop=True)
# df_diabetes_dn = df_diabetes.sample(n=458).reset_index(drop=True)
# df_others_dn = df_others.sample(n=489).reset_index(drop=True)
# df_myopia_dn = df_myopia.sample(n=478).reset_index(drop=True)
# df_amd_dn = df_amd.sample(n=469).reset_index(drop=True)
# df_glaucoma_dn = df_glaucoma.sample(n=494).reset_index(drop=True)
# # df_cataract_dn = df_cataract.sample(n=381).reset_index(drop=True)


In [67]:
# count labels
def extract_labels(df, initial):
    labels = []
    fundus_names = df.iloc[:]
    
    for i in range(len(fundus_names)):
        labels.append(initial)
        
    return labels

## Visualize Class Distribution (After data mapping)

In [68]:
total_count_labels = []
labels_each_count = []

labels_normal = extract_labels(df_norm_filenames_random, "N")
labels_diabetes = extract_labels(df_diabetes_dn, "D")
labels_glaucoma = extract_labels(df_glaucoma_dn, "G")
labels_cataract = extract_labels(df_cat_filenames, "C")
labels_amd = extract_labels(df_amd_dn, "A")
labels_hypertension = extract_labels(df_hypertension, "H")
labels_myopia = extract_labels(df_myopia_dn, "M")
labels_others = extract_labels(df_others_dn, "O")

total_count_labels += labels_normal
# total_count_labels += labels_diabetes
total_count_labels += labels_glaucoma
total_count_labels += labels_cataract
# total_count_labels += labels_amd
# total_count_labels += labels_hypertension
total_count_labels += labels_myopia
# total_count_labels += labels_others

labels_each_count.append(len(labels_normal))
# labels_each_count.append(len(labels_diabetes))
labels_each_count.append(len(labels_glaucoma))
labels_each_count.append(len(labels_cataract))
# labels_each_count.append(len(labels_amd))
# labels_each_count.append(len(labels_hypertension))
labels_each_count.append(len(labels_myopia))
# labels_each_count.append(len(labels_others))

print("Total Count: ", len(total_count_labels))
labels_each_count

In [69]:
init_notebook_mode(connected=False)
configure_plotly_browser_state()

labels_ser = pd.Series(total_count_labels, name='label')
labels_df = pd.DataFrame({
    'labels': labels_ser.value_counts().index,
    'count': labels_ser.value_counts()
})

labels_count = sorted(labels_each_count, reverse=True)
labels_df['percentage'] = [float(i / sum(labels_count)) for i in labels_count]

fig = plt.figure(figsize=(16, 16))
fig = go.Figure(data=[go.Bar(
                x=labels_df['labels'],
                y=labels_df['count'],
                text=labels_df['percentage'].apply(lambda x: "{0:1.2f}%".format(x * 100)),
                textposition='auto',
                marker=dict(
                    colorscale='Viridis'
                )
                )])

fig.update_layout(
    title='Resampled Fundus Image Distribution',
    xaxis_title='disease',
    yaxis_title='count',
    font=dict(
        size=13
    )
)

fig.show()

#### Split Data (After resampling)

In [70]:
# check for non-existent

# not_exists = []

# for file in tqdm(dframe_fundus.iloc[:]['Left-Fundus']):
#   if not os.path.exists(os.path.join(prep_train, file)):
#     not_exists.append(file)

# for file in tqdm(dframe_fundus.iloc[:]['Right-Fundus']):
#   if not os.path.exists(os.path.join(prep_train, file)):
#     not_exists.append(file)

# dframe_fundus = dframe_fundus.loc[(~dframe_fundus['Left-Fundus'].str.match('|'.join(not_exists)))]
# dframe_fundus = dframe_fundus.loc[(~dframe_fundus['Right-Fundus'].str.match('|'.join(not_exists)))]

# len(dframe_fundus)

In [71]:
left = []
right = []

# dframe_fundus = pd.concat([df_norm_filenames_random, df_cat_filenames, df_diabetes_dn,
#                           df_glaucoma_dn, df_myopia_dn, df_hypertension, df_amd_dn,
#                           df_others_dn]).reset_index(drop=True)

dframe_fundus = pd.concat([df_norm_filenames_random, df_cat_filenames, df_glaucoma_dn, df_myopia_dn])

dframe_fundus = dframe_fundus.sample(frac=1).reset_index(drop=True)
# dframe_fundus = dframe_fundus.sample(n=2600).reset_index(drop=True)
# dframe_fundus.tail(5)

# check if all filenames exist


for file in tqdm(dframe_fundus.iloc[:]['Left-Fundus']):
    if os.path.isfile(os.path.join(prep_train, file)):
#         img = cv2.imread(os.path.join(prep_train, file), cv2.IMREAD_UNCHANGED)
        img = cv2.imread(os.path.join(prep_train, file), 0)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 1)
        left.append(img)
        
for file in tqdm(dframe_fundus.iloc[:]['Right-Fundus']):
    if os.path.isfile(os.path.join(prep_train, file)):
        img = cv2.imread(os.path.join(prep_train, file), 0)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 1)
        right.append(img)
        
X1 = np.array(left)
X2 = np.array(right)

In [72]:
# RGB

left1 = []
right1 = []
for file in tqdm(dframe_fundus.iloc[:]['Left-Fundus']):
    if os.path.isfile(os.path.join(prep_train, file)):
        img = cv2.imread(os.path.join(prep_train, file), cv2.IMREAD_UNCHANGED)
#         img = cv2.imread(os.path.join(prep_train, file), 0)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 3)
        left1.append(img)
        
for file in tqdm(dframe_fundus.iloc[:]['Right-Fundus']):
    if os.path.isfile(os.path.join(prep_train, file)):
        img = cv2.imread(os.path.join(prep_train, file), cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape(IMG_SIZE, IMG_SIZE, 3)
        right1.append(img)
        
X3 = np.array(left1)
X4 = np.array(right1)

In [73]:
from sklearn.preprocessing import OneHotEncoder

classes = list(dframe_fundus.iloc[:, 2:11].sum().index)
# y = np.array(dframe_fundus.iloc[:][classes])
y = np.array(dframe_fundus.iloc[:][['N', 'C', 'G', 'M']])
y = np.array(y)
y = np.concatenate((y, y))
print("Labels: " + str(y.shape))

**Train Test Splitting**

**Singular Inputs**

In [74]:
from sklearn.model_selection import train_test_split

X = np.concatenate((X1, X2))
# X_norm = X / 255

x_train, x_test, \
y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

x_train, x_valid, \
y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

X_RGB = np.concatenate((X3, X4))
# X_norm = X / 255

x_train1, x_test1, \
y_train1, y_test1 = train_test_split(X_RGB, y, test_size=0.20, random_state=0)

x_train1, x_valid1, \
y_train1, y_valid1 = train_test_split(x_train1, y_train1, test_size=0.10, random_state=42)


# Step 7: Performance Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [77]:
from tensorflow.keras.models import load_model

custom_objects = {"custom_activation": tf.nn.crelu}
# model = load_model(os.path.join(model_path, "odir_checkpoint.h5"), custom_objects=custom_objects)
# model1 = load_model(os.path.join(model_path, "odir_model-{0:.2f}".format(test_acc)))
# model = load_model(odir_rgb_model_path)
# model1.load_weights(model_path, "odir_model-{0:.2f}-bw".format(test_acc))
# model = load_model(odir_model_dir)

# model = tf.saved_model.load(model_path)
# model.summary()

for folder,subfolders, files in os.walk('odir_rgb_model_four-0.9549'):
    for file in files:
        print(os.path.join(folder, file))

In [78]:
import pickle

resnet_crelu_model_path = "odir_rgb_model_four-0.9549/trained_models/odir/odir_model-0.95"
resnet50_v2_model_path =  "fine_tunedModels/resnet50v2/trained_models/odir/resnet50_v2/odir_model-0.75"
inceptionv3_model_path = "fine_tunedModels/inceptionv3/trained_models/odir/inception_v3/odir_model-0.85"
mobilenetv2_model_path = "fine_tunedModels/mobilenetv2/trained_models/odir/mobilenet_v2/odir_model-0.87"
densenet169_model_path = "fine_tunedModels/densenet169/trained_models/odir/densenet_169/odir_model-0.92"

resnet_crelu_history = "odir_rgb_model_four-0.9549/trained_models/odir/odir_history.h5"
resnet50_history = "fine_tunedModels/resnet50v2/trained_models/odir/resnet50_v2/odir_history.h5"
densenet_history = "fine_tunedModels/densenet169/trained_models/odir/densenet_169/odir_history.h5"
inception_history = "fine_tunedModels/inceptionv3/trained_models/odir/inception_v3/odir_history.h5"
mobilenet_history = "fine_tunedModels/mobilenetv2/trained_models/odir/mobilenet_v2/odir_history.h5"

resnet_crelu_model = load_model(resnet_crelu_model_path)
resnet50_v2_model = load_model(resnet50_v2_model_path)
inceptionv3_model = load_model(inceptionv3_model_path)
mobilenetv2_model = load_model(mobilenetv2_model_path)
densenet169_model = load_model(densenet169_model_path)

resnet_crelu_pickle_in = open(resnet_crelu_history, "rb")
resnet50_v2_pickle_in = open(resnet50_history, "rb")
inceptionv3_pickle_in = open(inception_history, "rb")
densenet_pickle_in = open(densenet_history, "rb")
mobilenet_pickle_in = open(mobilenet_history, "rb")

resnet_crelu_saved_history = pickle.load(resnet_crelu_pickle_in)
resnet50_v2_saved_history = pickle.load(resnet50_v2_pickle_in)
inceptionv3_saved_history = pickle.load(inceptionv3_pickle_in)
densenet_saved_history = pickle.load(densenet_pickle_in)
mobilenet_saved_history = pickle.load(mobilenet_pickle_in)

resnet_crelu_pickle_in.close()
resnet50_v2_pickle_in.close()
inceptionv3_pickle_in.close()
densenet_pickle_in.close()
mobilenet_pickle_in.close()


In [79]:
_, rc_test_acc, rc_test_prec, rc_test_auc = resnet_crelu_model.evaluate(x_test1, y_test1)
_, r50_test_acc, r50_test_prec, r50_test_auc = resnet50_v2_model.evaluate(x_test1, y_test1)
_, inc_test_acc, inc_test_prec, inc_test_auc = inceptionv3_model.evaluate(x_test1, y_test1)
_, dense_test_acc, dense_test_prec, dense_test_auc = densenet169_model.evaluate(x_test1, y_test1)
_, mobile_test_acc, mobile_test_prec, mobile_test_auc = mobilenetv2_model.evaluate(x_test1, y_test1)

print("ResNet128 CReLU:")
print("Test Accuracy: {0:.2f}% - Test Precision: {1: .2f}% - Test AUC: {2: .2f}%".format(rc_test_acc * 100, rc_test_prec * 100, rc_test_auc * 100))
print('-'*25)
print("ResNet50V2:")
print("Test Accuracy: {0:.2f}% - Test Precision: {1: .2f}% - Test AUC: {2: .2f}%".format(r50_test_acc * 100, r50_test_prec * 100, r50_test_auc * 100))
print('-'*25)
print("InceptionV3:")
print("Test Accuracy: {0:.2f}% - Test Precision: {1: .2f}% - Test AUC: {2: .2f}%".format(inc_test_acc * 100, inc_test_prec * 100, inc_test_auc * 100))
print('-'*25)
print("MobileNetV2:")
print("Test Accuracy: {0:.2f}% - Test Precision: {1: .2f}% - Test AUC: {2: .2f}%".format(mobile_test_acc * 100, mobile_test_prec * 100, mobile_test_auc * 100))
print('-'*25)
print('DenseNet169:')
print("Test Accuracy: {0:.2f}% - Test Precision: {1: .2f}% - Test AUC: {2: .2f}%".format(dense_test_acc * 100, dense_test_prec * 100, dense_test_auc * 100))

model_evaluation_df = pd.DataFrame({
    "accuracy": [rc_test_acc, r50_test_acc, inc_test_acc, dense_test_acc, mobile_test_acc],
    "precision": [rc_test_acc, r50_test_acc, inc_test_acc, mobile_test_acc, dense_test_acc],
    "AUC": [rc_test_auc, r50_test_auc, inc_test_auc, mobile_test_auc, dense_test_auc]
})

model_evaluation_df.head()

In [96]:
import plotly.graph_objs as go
# define scatter plot visualization on plotly express

NUM_EPOCHS = 150
def display_training_curves(rc_train, r50_train, inc_train, dense_train, mobile_train, yaxis):
    if yaxis == "loss":
        ylabel = "Loss"
        title = "Model Loss with respect to Epochs"
    elif yaxis == "accuracy":
        ylabel = "Accuracy"
        title = "Model Accuracy with respect to Epochs"
    elif yaxis == "precision":
        ylabel = "Precision"
        title = "Model Precision with respect to Epochs"
    elif yaxis == "auc":
        ylabel = "AUC"
        title = "Model AUC with respect to Epochs"
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(x = np.arange(1, NUM_EPOCHS + 1, 1), mode='lines+markers',
                  y=rc_train, marker=dict(color='#636ef1'), name='ResNet BN-CReLU')
    )

    fig.add_trace(
        go.Scatter(x = np.arange(1, NUM_EPOCHS + 1, 1), mode='lines+markers',
                  y=r50_train, marker=dict(color='#00cc96'), name='ResNet50V2')
    )

    fig.add_trace(
        go.Scatter(x = np.arange(1, NUM_EPOCHS + 1, 1), mode='lines+markers',
                  y=inc_train, marker=dict(color='#19d3f3'), name='InceptionV3')
    )

    fig.add_trace(
        go.Scatter(x = np.arange(1, NUM_EPOCHS + 1, 1), mode='lines+markers',
                  y=mobile_train, marker=dict(color='#ffa15a'), name='MobileNetV2')
    )

    fig.add_trace(
        go.Scatter(x = np.arange(1, NUM_EPOCHS + 1, 1), mode='lines+markers',
                  y=dense_train, marker=dict(color='#ff6692'), name='DenseNet169')
    )
    
    # fig.add_trace(
    #     go.Scatter(x=np.arange(1, NUM_EPOCHS + 1, 1), mode='lines+markers', y=validation, marker=dict(color='darkorange'),
    #               name='Validation')
    # )
    
    fig.update_layout(title=title, yaxis_title=ylabel,
                     xaxis_title='Epochs', template='plotly_white')
    fig.show()

In [147]:
plt.style.use('ggplot')

fig1, ax = plt.subplots(4, 1)
plt.subplots_adjust(wspace=0, hspace=0.2)

line1 = ax[0].plot(resnet_crelu_saved_history['val_accuracy'], label='ResNet-BN_CReLU Accuracy')
line2 = ax[0].plot(resnet50_v2_saved_history['val_accuracy'], label="ResNet50V2 Accuracy")
line3 = ax[0].plot(inceptionv3_saved_history['val_accuracy'], label="InceptionV3 Accuracy")
line4 = ax[0].plot(mobilenet_saved_history['val_accuracy'], label="MobileNetV2 Accuracy")
line5 = ax[0].plot(densenet_saved_history['val_accuracy'], label="DenseNet169 Accuracy")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.setp(line3, linewidth=2.0)
plt.setp(line4, linewidth=2.0)
plt.setp(line5, linewidth=2.0)
ax[0].set_title("Model Accuracy")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

line1 = ax[1].plot(resnet_crelu_saved_history['val_loss'], label='ResNet-BN_CReLU Loss')
line2 = ax[1].plot(resnet50_v2_saved_history['val_loss'], label="ResNet50V2 Loss")
line3 = ax[1].plot(inceptionv3_saved_history['val_loss'], label="InceptionV3 Loss")
line4 = ax[1].plot(mobilenet_saved_history['val_loss'], label="MobileNetV2 Loss")
line5 = ax[1].plot(densenet_saved_history['val_loss'], label="DenseNet169 Loss")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.setp(line3, linewidth=2.0)
plt.setp(line4, linewidth=2.0)
plt.setp(line5, linewidth=2.0)
ax[1].set_title("Model Loss")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")

line1 = ax[2].plot(resnet_crelu_saved_history['val_precision'], label='ResNet-BN_CReLU Precision')
line2 = ax[2].plot(resnet50_v2_saved_history['val_precision'], label="ResNet50V2 Precision")
line3 = ax[2].plot(inceptionv3_saved_history['val_precision'], label="InceptionV3 Precision")
line4 = ax[2].plot(mobilenet_saved_history['val_precision'], label="MobileNetV2 Precision")
line5 = ax[2].plot(densenet_saved_history['val_precision'], label="DenseNet169 Precision")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.setp(line3, linewidth=2.0)
plt.setp(line4, linewidth=2.0)
plt.setp(line5, linewidth=2.0)
ax[2].set_title("Model Precision")
ax[2].set_xlabel("Epochs")
ax[2].set_ylabel("Precision")

line1 = ax[3].plot(resnet_crelu_saved_history['val_auc'], label='ResNet-BN_CReLU AUC')
line2 = ax[3].plot(resnet50_v2_saved_history['val_auc'], label="ResNet50V2 AUC")
line3 = ax[3].plot(inceptionv3_saved_history['val_auc'], label="InceptionV3 AUC")
line4 = ax[3].plot(mobilenet_saved_history['val_auc'], label="MobileNetV2 AUC")
line5 = ax[3].plot(densenet_saved_history['val_auc'], label="DenseNet169 AUC")
plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.setp(line3, linewidth=2.0)
plt.setp(line4, linewidth=2.0)
plt.setp(line5, linewidth=2.0)
ax[3].set_title("Model AUC")
ax[3].set_xlabel("Epochs")
ax[3].set_ylabel("AUC")

for i in range(4):
  ax[i].legend(loc="lower left")

fig1.canvas.set_window_title("Model Comparison")
fig1.set_figwidth(12)
fig1.set_figheight(8)

plt.tight_layout()
plt.show()

In [84]:
plt.style.use('ggplot')

fig1, ax = plt.subplots(1, 1)
plt.subplots_adjust(wspace=0, hspace=0.2)

line1 = ax.plot(resnet_crelu_saved_history['val_accuracy'], label='ResNet BN-CReLU Accuracy')
line2 = ax.plot(resnet50_v2_saved_history['val_accuracy'], label='ResNet50V2 Accuracy')
line3 = ax.plot(inceptionv3_saved_history['val_accuracy'], label='InceptionV3 Accuracy')
line4 = ax.plot(mobilenet_saved_history['val_accuracy'], label='MobileNetV2 Accuracy')
line5 = ax.plot(densenet_saved_history['val_accuracy'], label='DenseNet169 Accuracy')

plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.setp(line3, linewidth=2.0)
plt.setp(line4, linewidth=2.0)
plt.setp(line5, linewidth=2.0)

ax.legend()

fig1.canvas.set_window_title("Model Evaluation on Accuracy")
fig1.set_figwidth(12)
fig1.set_figheight(3)

plt.tight_layout()
plt.show()

**Model Accuracy**

In [97]:
display_training_curves(
    resnet_crelu_saved_history['val_accuracy'],
    resnet50_v2_saved_history['val_accuracy'],
    inceptionv3_saved_history['val_accuracy'],
    mobilenet_saved_history['val_accuracy'],
    densenet_saved_history['val_accuracy'],
    'accuracy'
)

**Model Loss**

In [98]:
plt.style.use('ggplot')

fig1, ax = plt.subplots(1, 1)
plt.subplots_adjust(wspace=0, hspace=0.2)

line1 = ax.plot(resnet_crelu_saved_history['val_loss'], label='ResNet BN-CReLU Loss')
line2 = ax.plot(resnet50_v2_saved_history['val_loss'], label='ResNet50V2 Loss')
line3 = ax.plot(inceptionv3_saved_history['val_loss'], label='InceptionV3 Loss')
line4 = ax.plot(mobilenet_saved_history['val_loss'], label='MobileNetV2 Loss')
line5 = ax.plot(densenet_saved_history['val_loss'], label='DenseNet169 Loss')

plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.setp(line3, linewidth=2.0)
plt.setp(line4, linewidth=2.0)
plt.setp(line5, linewidth=2.0)

ax.legend()

fig1.canvas.set_window_title("Model Evaluation on Loss")
fig1.set_figwidth(12)
fig1.set_figheight(3)

plt.tight_layout()
plt.show()

In [99]:
display_training_curves(
    resnet_crelu_saved_history['val_loss'],
    resnet50_v2_saved_history['val_loss'],
    inceptionv3_saved_history['val_loss'],
    mobilenet_saved_history['val_loss'],
    densenet_saved_history['val_loss'],
    'loss'
)

**Model Precision**

In [148]:
plt.style.use('ggplot')

fig1, ax = plt.subplots(1, 1)
plt.subplots_adjust(wspace=0, hspace=0.2)

line1 = ax.plot(resnet_crelu_saved_history['val_precision'], label='ResNet BN-CReLU Precision')
line2 = ax.plot(resnet50_v2_saved_history['val_precision'], label='ResNet50V2 Precision')
line3 = ax.plot(inceptionv3_saved_history['val_precision'], label='InceptionV3 Precision')
line4 = ax.plot(mobilenet_saved_history['val_precision'], label='MobileNetV2 Precision')
line5 = ax.plot(densenet_saved_history['val_precision'], label='DenseNet169 Precision')

plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.setp(line3, linewidth=2.0)
plt.setp(line4, linewidth=2.0)
plt.setp(line5, linewidth=2.0)

ax.legend()

fig1.canvas.set_window_title("Model Evaluation on Precision")
fig1.set_figwidth(12)
fig1.set_figheight(3)

plt.tight_layout()
plt.show()

In [101]:
display_training_curves(
    resnet_crelu_saved_history['val_precision'],
    resnet50_v2_saved_history['val_precision'],
    inceptionv3_saved_history['val_precision'],
    mobilenet_saved_history['val_precision'],
    densenet_saved_history['val_precision'],
    'precision'
)

**Model AUC**

In [102]:
plt.style.use('ggplot')

fig1, ax = plt.subplots(1, 1)
plt.subplots_adjust(wspace=0, hspace=0.2)

line1 = ax.plot(resnet_crelu_saved_history['val_auc'], label='ResNet BN-CReLU AUC')
line2 = ax.plot(resnet50_v2_saved_history['val_auc'], label='ResNet50V2 AUC')
line3 = ax.plot(inceptionv3_saved_history['val_auc'], label='InceptionV3 AUC')
line4 = ax.plot(mobilenet_saved_history['val_auc'], label='MobileNetV2 AUC')
line5 = ax.plot(densenet_saved_history['val_auc'], label='DenseNet169 AUC')

plt.setp(line1, linewidth=2.0)
plt.setp(line2, linewidth=2.0)
plt.setp(line3, linewidth=2.0)
plt.setp(line4, linewidth=2.0)
plt.setp(line5, linewidth=2.0)

ax.legend()

fig1.canvas.set_window_title("Model Evaluation on AUC")
fig1.set_figwidth(12)
fig1.set_figheight(3)

plt.tight_layout()
plt.show()

In [104]:
display_training_curves(
    resnet_crelu_saved_history['val_auc'],
    resnet50_v2_saved_history['val_auc'],
    inceptionv3_saved_history['val_auc'],
    mobilenet_saved_history['val_auc'],
    densenet_saved_history['val_auc'],
    'auc'
)

## Step 7.1: Confusion Matrix and Classification Report

In [ ]:
# class_labels = list(train_generator.class_indices.keys())
# print(class_labels)

In [105]:
# class_labels = list(annot_df.iloc[:, -12:-4].sum().index)
class_labels = ['N', 'C', 'G', 'M']
# class_labels = classes

In [ ]:
# resnet_crelu_model = load_model(resnet_crelu_model_path)
# resnet50_v2_model = load_model(resnet50_v2_model_path)
# inceptionv3_model = load_model(inceptionv3_model_path)
# mobilenetv2_model = load_model(mobilenetv2_model_path)
# densenet169_model = load_model(densenet169_model_path)

In [150]:
# y_pred_raw = model.predict_generator(test_generator, verbose=1)
y_pred_raw = resnet_crelu_model.predict(x_test1, verbose=1)
y_pred = np.argmax(y_pred_raw, axis=1)

y_test_classes = np.argmax(y_test1, axis=1)

# print("Confusion Matrix")
# print(confusion_matrix(test_generator.classes, y_pred))
print("ResNet BN CReLU Classfication Report")
print(classification_report(y_test_classes, y_pred, target_names=class_labels))

print('-'*60)

y_pred_raw = resnet50_v2_model.predict(x_test1, verbose=1)
y_pred = np.argmax(y_pred_raw, axis=1)

y_test_classes = np.argmax(y_test1, axis=1)

# print("Confusion Matrix")
# print(confusion_matrix(test_generator.classes, y_pred))
print("ResNet50V2 Classfication Report")
print(classification_report(y_test_classes, y_pred, target_names=class_labels))

print('-'*60)

y_pred_raw = inceptionv3_model.predict(x_test1, verbose=1)
y_pred = np.argmax(y_pred_raw, axis=1)

y_test_classes = np.argmax(y_test1, axis=1)

# print("Confusion Matrix")
# print(confusion_matrix(test_generator.classes, y_pred))
print("InceptionV3 Classfication Report")
print(classification_report(y_test_classes, y_pred, target_names=class_labels))

print('-'*60)
y_pred_raw = mobilenetv2_model.predict(x_test1, verbose=1)
y_pred = np.argmax(y_pred_raw, axis=1)

y_test_classes = np.argmax(y_test1, axis=1)

# print("Confusion Matrix")
# print(confusion_matrix(test_generator.classes, y_pred))
print("MobileNetV2 Classfication Report")
print(classification_report(y_test_classes, y_pred, target_names=class_labels))

print('-'*60)

y_pred_raw = densenet169_model.predict(x_test1, verbose=1)
y_pred = np.argmax(y_pred_raw, axis=1)

y_test_classes = np.argmax(y_test1, axis=1)

# print("Confusion Matrix")
# print(confusion_matrix(test_generator.classes, y_pred))
print("DenseNet169 Classfication Report")
print(classification_report(y_test_classes, y_pred, target_names=class_labels))

**Models Conf Matrix**

In [115]:
rc_y_pred_raw = resnet_crelu_model.predict(x_test1, verbose=1)
rc_y_pred = np.argmax(rc_y_pred_raw, axis=1)
rc_y_test_classes = np.argmax(y_test1, axis=1)

plt.figure(figsize=(14, 14))
# cm = confusion_matrix(test_generator.classes, y_pred)
rc_conf_mtrx = confusion_matrix(rc_y_test_classes, rc_y_pred)
rc_cm = np.around(rc_conf_mtrx.astype('float') / rc_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)

threshold = rc_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(rc_cm.shape[1]), range(rc_cm.shape[0])):
  color = 'black' if rc_cm[i, j] > threshold else "white"
  plt.text(j, i, rc_cm[i, j], horizontalalignment='center', color=color)
    
plt.imshow(rc_cm)
plt.title("ResNet BN-CReLUConfusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

In [116]:
r50_y_pred_raw = resnet50_v2_model.predict(x_test1, verbose=1)
r50_y_pred = np.argmax(r50_y_pred_raw, axis=1)
r50_y_test_classes = np.argmax(y_test1, axis=1)

r50_conf_mtrx = confusion_matrix(r50_y_test_classes, r50_y_pred)
r50_cm = np.around(r50_conf_mtrx.astype('float') / r50_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)

threshold = 0    
threshold = r50_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(r50_cm.shape[1]), range(r50_cm.shape[0])):
  color = 'black' if r50_cm[i, j] > threshold else "white"
  plt.text(j, i, r50_cm[i, j], horizontalalignment='center', color=color)
    
plt.imshow(r50_cm)
plt.title("ResNet50V2 Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

In [151]:
inc_y_pred_raw = inceptionv3_model.predict(x_test1, verbose=1)
inc_y_pred = np.argmax(inc_y_pred_raw, axis=1)
inc_y_test_classes = np.argmax(y_test1, axis=1)

inc_conf_mtrx = confusion_matrix(inc_y_test_classes, inc_y_pred)
inc_cm = np.around(inc_conf_mtrx.astype('float') / inc_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)

threshold = 0    
threshold = inc_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(inc_cm.shape[1]), range(inc_cm.shape[0])):
  color = 'black' if inc_cm[i, j] > threshold else "white"
  plt.text(j, i, inc_cm[i, j], horizontalalignment='center', color=color)

plt.imshow(inc_cm)
plt.title("InceptionV3 Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

In [117]:
mobile_y_pred_raw = mobilenetv2_model.predict(x_test1, verbose=1)
mobile_y_pred = np.argmax(mobile_y_pred_raw, axis=1)
mobile_y_test_classes = np.argmax(y_test1, axis=1)

mobile_conf_mtrx = confusion_matrix(mobile_y_test_classes, mobile_y_pred)
mobile_cm = np.around(mobile_conf_mtrx.astype('float') / mobile_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)

threshold = 0
threshold = mobile_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(mobile_cm.shape[1]), range(mobile_cm.shape[0])):
  color = 'black' if mobile_cm[i, j] > threshold else "white"
  plt.text(j, i, mobile_cm[i, j], horizontalalignment='center', color=color)
    
plt.imshow(mobile_cm)
plt.title("MobileNetV2 Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

In [118]:
dense_y_pred_raw = densenet169_model.predict(x_test1, verbose=1)
dense_y_pred = np.argmax(dense_y_pred_raw, axis=1)
dense_y_test_classes = np.argmax(y_test1, axis=1)

dense_conf_mtrx = confusion_matrix(dense_y_test_classes, dense_y_pred)
dense_cm = np.around(dense_conf_mtrx.astype('float') / dense_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)

threshold = 0
threshold = dense_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(dense_cm.shape[1]), range(dense_cm.shape[0])):
  color = 'black' if dense_cm[i, j] > threshold else "white"
  plt.text(j, i, dense_cm[i, j], horizontalalignment='center', color=color)
    
plt.imshow(dense_cm)
plt.title("DenseNet169 Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

In [114]:
# y_pred_raw = model.predict_generator(test_generator, verbose=1)
# y_pred_raw = model.predict(x_test, verbose=1)
# y_pred = np.argmax(y_pred_raw, axis=1)

# nb_test_samples = test_generator.samples
# nb_train_samples = train_generator.samples
rc_y_pred_raw = resnet_crelu_model.predict(x_test1, verbose=1)
r50_y_pred_raw = resnet50_v2_model.predict(x_test1, verbose=1)
inc_y_pred_raw = inceptionv3_model.predict(x_test1, verbose=1)
mobile_y_pred_raw = mobilenetv2_model.predict(x_test1, verbose=1)
dense_y_pred_raw = densenet169_model.predict(x_test1, verbose=1)

rc_y_pred = np.argmax(rc_y_pred_raw, axis=1)
r50_y_pred = np.argmax(r50_y_pred_raw, axis=1)
inc_y_pred = np.argmax(inc_y_pred_raw, axis=1)
mobile_y_pred = np.argmax(mobile_y_pred_raw, axis=1)
dense_y_pred = np.argmax(dense_y_pred_raw, axis=1)

rc_y_test_classes = np.argmax(y_test1, axis=1)
r50_y_test_classes = np.argmax(y_test1, axis=1)
inc_y_test_classes = np.argmax(y_test1, axis=1)
mobile_y_test_classes = np.argmax(y_test1, axis=1)
dense_y_test_classes = np.argmax(y_test1, axis=1)

plt.figure(figsize=(14, 14))
# cm = confusion_matrix(test_generator.classes, y_pred)
rc_conf_mtrx = confusion_matrix(rc_y_test_classes, rc_y_pred)
r50_conf_mtrx = confusion_matrix(r50_y_test_classes, r50_y_pred)
inc_conf_mtrx = confusion_matrix(inc_y_test_classes, inc_y_pred)
mobile_conf_mtrx = confusion_matrix(mobile_y_test_classes, mobile_y_pred)
dense_conf_mtrx = confusion_matrix(dense_y_test_classes, dense_y_pred)

rc_cm = np.around(rc_conf_mtrx.astype('float') / rc_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)
r50_cm = np.around(r50_conf_mtrx.astype('float') / r50_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)
inc_cm = np.around(inc_conf_mtrx.astype('float') / inc_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)
mobile_cm = np.around(mobile_conf_mtrx.astype('float') / mobile_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)
dense_cm = np.around(dense_conf_mtrx.astype('float') / dense_conf_mtrx.sum(axis=1)[:, np.newaxis], decimals=2)


# use white text if squares are dark; otherwise black
threshold = 0
threshold = rc_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(rc_cm.shape[1]), range(rc_cm.shape[0])):
  color = 'black' if rc_cm[i, j] > threshold else "white"
  plt.text(j, i, rc_cm[i, j], horizontalalignment='center', color=color)

threshold = 0    
threshold = r50_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(r50_cm.shape[1]), range(r50_cm.shape[0])):
  color = 'black' if r50_cm[i, j] > threshold else "white"
  plt.text(j, i, r50_cm[i, j], horizontalalignment='center', color=color)

threshold = 0    
threshold = inc_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(inc_cm.shape[1]), range(inc_cm.shape[0])):
  color = 'black' if inc_cm[i, j] > threshold else "white"
  plt.text(j, i, inc_cm[i, j], horizontalalignment='center', color=color)

threshold = 0
threshold = mobile_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(mobile_cm.shape[1]), range(mobile_cm.shape[0])):
  color = 'black' if mobile_cm[i, j] > threshold else "white"
  plt.text(j, i, mobile_cm[i, j], horizontalalignment='center', color=color)

threshold = 0
threshold = dense_cm.max() / 2
plt.rcParams.update({'font.size': 15})
for i, j in itertools.product(range(dense_cm.shape[1]), range(dense_cm.shape[0])):
  color = 'black' if dense_cm[i, j] > threshold else "white"
  plt.text(j, i, dense_cm[i, j], horizontalalignment='center', color=color)

plt.imshow(rc_cm)
plt.title("ResNet BN-CReLUConfusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

print('-'*60)

plt.imshow(r50_cm)
plt.title("ResNet50V2 Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

print('-'*60)

plt.imshow(inc_cm)
plt.title("InceptionV3 Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

print('-'*60)

plt.imshow(mobile_cm)
plt.title("MobileNetV2 Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

print('-'*60)

plt.imshow(dense_cm)
plt.title("DenseNet169 Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=90)
plt.yticks(tick_marks, class_labels)

plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

print('-'*60)


In [ ]:
print(conf_mtrx)

**Compute Kappa**

In [152]:
# agreed = (conf_mtrx[0][0] + conf_mtrx[1][1] + conf_mtrx[2][2] + conf_mtrx[3][3]) / len(y_test)
# prob_true_N = float((conf_mtrx[0][0] + conf_mtrx[1][0] + conf_mtrx[2][0] + conf_mtrx[3][0]) / len(y_test))
# prob_pred_N = float((conf_mtrx[0][0] + conf_mtrx[0][1] + conf_mtrx[0][2] + conf_mtrx[0][3]) / len(y_test))
# chance_N = float(prob_true_N * prob_pred_N)

# prob_true_C = float((conf_mtrx[0][1] + conf_mtrx[1][1] + conf_mtrx[2][1] + conf_mtrx[3][1]) / len(y_test))
# prob_pred_C = float((conf_mtrx[1][0] + conf_mtrx[1][1] + conf_mtrx[1][2] + conf_mtrx[1][3]) / len(y_test))
# chance_C = float(prob_true_C * prob_pred_C)

# prob_true_G = float((conf_mtrx[0][2] + conf_mtrx[1][2] + conf_mtrx[2][2] + conf_mtrx[3][2]) / len(y_test))
# prob_pred_G = float((conf_mtrx[2][0] + conf_mtrx[2][1] + conf_mtrx[2][2] + conf_mtrx[2][3]) / len(y_test))
# chance_G = float(prob_true_G * prob_pred_G)

# prob_true_M = float((conf_mtrx[0][3] + conf_mtrx[1][3] + conf_mtrx[2][3] + conf_mtrx[3][3]) / len(y_test))
# prob_pred_M = float((conf_mtrx[3][0] + conf_mtrx[3][1] + conf_mtrx[3][2] + conf_mtrx[3][3]) / len(y_test))
# chance_M = float(prob_true_M * prob_pred_M)

# chance_agreed = float(chance_N + chance_C + chance_G + chance_M)

# kappa_score = float((agreed - chance_agreed)/(1-chance_agreed))
# print("Kappa Score: {0:.4f}".format(kappa_score))

In [ ]:
agreed

**RC Class Evaluation**

In [145]:
print("-----ResNet BN CReLU Class Evaluation-----")
# Normal Evaluation
N_tp = rc_conf_mtrx[0][0]
N_fn = (rc_conf_mtrx[1][0] + rc_conf_mtrx[2][0] + rc_conf_mtrx[3][0])
N_tn = (rc_conf_mtrx[1][1] + rc_conf_mtrx[1][2] + rc_conf_mtrx[1][3] + \
        rc_conf_mtrx[2][1] + rc_conf_mtrx[2][2] + rc_conf_mtrx[2][3] + \
        rc_conf_mtrx[3][1] + rc_conf_mtrx[3][2] + rc_conf_mtrx[3][3])
N_fp = (rc_conf_mtrx[0][1] + rc_conf_mtrx[0][2] + rc_conf_mtrx[0][3])

N_tpr = N_tp / (N_tp + N_fn)
N_tnr = N_tn / (N_tn + N_fp)
N_fpr = N_fp / (N_fp + N_tn)
N_fnr = N_fn / (N_fn + N_tp)

print("Normal Category: ")
print("-->  True Positives: ", N_tp)
print("-->  True Negatives: ", N_tn)
print("-->  False Positives: ", N_fp)
print("-->  False Negatives: ", N_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(N_tpr))
print("-->  True Negative Rate: {0:.4f}".format(N_tnr))
print("-->  False Positive Rate: {0:.4f}".format(N_fpr))
print("-->  False Negative Rate: {0:.4f}".format(N_fnr))

print()
print('=*=*'*8)
print()

# Cataract Evaluation
C_tp = rc_conf_mtrx[1][1]
C_tn = (rc_conf_mtrx[0][0] + rc_conf_mtrx[0][2] + rc_conf_mtrx[0][3] + \
        rc_conf_mtrx[2][0] + rc_conf_mtrx[2][2] + rc_conf_mtrx[2][3] + \
        rc_conf_mtrx[3][0] + rc_conf_mtrx[3][2] + rc_conf_mtrx[3][3])
C_fp = (rc_conf_mtrx[1][0] + rc_conf_mtrx[1][2] + rc_conf_mtrx[1][3])
C_fn = (rc_conf_mtrx[0][1] + rc_conf_mtrx[2][1] + rc_conf_mtrx[3][1])

C_tpr = C_tp / (C_tp + C_fn)
C_tnr = C_tn / (C_tn + C_fp)
C_fpr = C_fp / (C_fp + C_tn)
C_fnr = C_fn / (C_fn + C_tp)

print("Cataract Category: ")
print("-->  True Positives: ", C_tp)
print("-->  True Negatives: ", C_tn)
print("-->  False Positives: ", C_fp)
print("-->  False Negatives: ", C_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(C_tpr))
print("-->  True Negative Rate: {0:.4f}".format(C_tnr))
print("-->  False Positive Rate: {0:.4f}".format(C_fpr))
print("-->  False Negative Rate: {0:.4f}".format(C_fnr))

print()
print('=*=*'*8)
print()

# Glaucoma Evaluation
G_tp = rc_conf_mtrx[2][2]
G_tn = (rc_conf_mtrx[0][0] + rc_conf_mtrx[0][1] + rc_conf_mtrx[0][3] + \
        rc_conf_mtrx[1][0] + rc_conf_mtrx[1][1] + rc_conf_mtrx[1][3] + \
        rc_conf_mtrx[3][0] + rc_conf_mtrx[3][1] + rc_conf_mtrx[3][3])
G_fp = (rc_conf_mtrx[2][0] + rc_conf_mtrx[2][1] + rc_conf_mtrx[2][3])
G_fn = (rc_conf_mtrx[0][2] + rc_conf_mtrx[1][2] + rc_conf_mtrx[3][2])

G_tpr = G_tp / (G_tp + G_fn)
G_tnr = G_tn / (G_tn + G_fp)
G_fpr = G_fp / (G_fp + G_tn)
G_fnr = G_fn / (G_fn + G_tp)

print("Glaucoma Category: ")
print("-->  True Positives: ", G_tp)
print("-->  True Negatives: ", G_tn)
print("-->  False Positives: ", G_fp)
print("-->  False Negatives: ", G_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(G_tpr))
print("-->  True Negative Rate: {0:.4f}".format(G_tnr))
print("-->  False Positive Rate: {0:.4f}".format(G_fpr))
print("-->  False Negative Rate: {0:.4f}".format(G_fnr))

print()
print('=*=*'*8)
print()

# Myopia Evaluation
M_tp = rc_conf_mtrx[3][3]
M_tn = (rc_conf_mtrx[0][0] + rc_conf_mtrx[0][1] + rc_conf_mtrx[0][2] + \
        rc_conf_mtrx[1][0] + rc_conf_mtrx[1][1] + rc_conf_mtrx[1][2] + \
        rc_conf_mtrx[2][0] + rc_conf_mtrx[2][1] + rc_conf_mtrx[2][2])
M_fp = (rc_conf_mtrx[3][0] + rc_conf_mtrx[3][1] + rc_conf_mtrx[3][2])
M_fn = (rc_conf_mtrx[0][3] + rc_conf_mtrx[1][3] + rc_conf_mtrx[2][3])

M_tpr = M_tp / (M_tp + M_fn)
M_tnr = M_tn / (M_tn + M_fp)
M_fpr = M_fp / (M_fp + M_tn)
M_fnr = M_fn / (M_fn + M_tp)

print("Myopia Category: ")
print("-->  True Positives: ", M_tp)
print("-->  True Negatives: ", M_tn)
print("-->  False Positives: ", M_fp)
print("-->  False Negatives: ", M_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(M_tpr))
print("-->  True Negative Rate: {0:.4f}".format(M_tnr))
print("-->  False Positive Rate: {0:.4f}".format(M_fpr))
print("-->  False Negative Rate: {0:.4f}".format(M_fnr))


**RC Confusion Metric**

In [124]:
def confusion_metrics(cm):
#     TP = cm[1][0]
#     TN = cm[0][1]
#     FP = cm[0][0]
#     FN = cm[1][1]

    TP = (N_tp + C_tp + G_tp + M_tp)
    TN = (N_tn + C_tn + G_tn + M_tn)
    FP = (N_fp + C_fp + G_fp + M_fp)
    FN = (N_fn + C_fn + G_fn + M_fn)
    
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
    
    print("True Positives: ", TP)
    print("True Negatives: ", TN)
    print("False Positives: ", FP)
    print("False Negatives: ", FN)
    print('-'*35)
    print("True Positive Rate: {0:.4f}".format(TPR))
    print("True Negative Rate: {0:.4f}".format(TNR))
    print("False Positive Rate: {0:.4f}".format(FPR))
    print("False Negative Rate: {0:.4f}".format(FNR))
    
    # calculate accuracy
    conf_accuracy = (float(TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1 - conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    
    # calculate specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))

    # calculate f1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    
    print('-'*50)
    print(f"Accuracy: {round(conf_accuracy, 4)}")
    print(f"Mis-Classification: {round(conf_misclassification, 4)}")
    print(f"Sensitivity: {round(conf_sensitivity, 4)}")
    print(f"Specificity: {round(conf_specificity,4)}")
    print(f"G-Mean: {round(np.sqrt(round(conf_sensitivity,4) * round(conf_specificity, 4)), 4)}")
    print(f"Precision: {round(conf_precision, 4)}")
    print(f"F1 Score: {round(conf_f1, 4)}")

print("ResNet BN CReLU Confusion Metrics")
confusion_metrics(rc_conf_mtrx)

In [125]:
print("-----ResNet50V2 Class Evaluation-----")
# Normal Evaluation
N_tp = r50_conf_mtrx[0][0]
N_fn = (r50_conf_mtrx[1][0] + r50_conf_mtrx[2][0] + r50_conf_mtrx[3][0])
N_tn = (r50_conf_mtrx[1][1] + r50_conf_mtrx[1][2] + r50_conf_mtrx[1][3] + \
        r50_conf_mtrx[2][1] + r50_conf_mtrx[2][2] + r50_conf_mtrx[2][3] + \
        r50_conf_mtrx[3][1] + r50_conf_mtrx[3][2] + r50_conf_mtrx[3][3])
N_fp = (r50_conf_mtrx[0][1] + r50_conf_mtrx[0][2] + r50_conf_mtrx[0][3])

N_tpr = N_tp / (N_tp + N_fn)
N_tnr = N_tn / (N_tn + N_fp)
N_fpr = N_fp / (N_fp + N_tn)
N_fnr = N_fn / (N_fn + N_tp)

print("Normal Category: ")
print("-->  True Positives: ", N_tp)
print("-->  True Negatives: ", N_tn)
print("-->  False Positives: ", N_fp)
print("-->  False Negatives: ", N_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(N_tpr))
print("-->  True Negative Rate: {0:.4f}".format(N_tnr))
print("-->  False Positive Rate: {0:.4f}".format(N_fpr))
print("-->  False Negative Rate: {0:.4f}".format(N_fnr))

print()
print('=*=*'*8)
print()

# Cataract Evaluation
C_tp = r50_conf_mtrx[1][1]
C_tn = (r50_conf_mtrx[0][0] + r50_conf_mtrx[0][2] + r50_conf_mtrx[0][3] + \
        r50_conf_mtrx[2][0] + r50_conf_mtrx[2][2] + r50_conf_mtrx[2][3] + \
        r50_conf_mtrx[3][0] + r50_conf_mtrx[3][2] + r50_conf_mtrx[3][3])
C_fp = (r50_conf_mtrx[1][0] + r50_conf_mtrx[1][2] + r50_conf_mtrx[1][3])
C_fn = (r50_conf_mtrx[0][1] + r50_conf_mtrx[2][1] + r50_conf_mtrx[3][1])

C_tpr = C_tp / (C_tp + C_fn)
C_tnr = C_tn / (C_tn + C_fp)
C_fpr = C_fp / (C_fp + C_tn)
C_fnr = C_fn / (C_fn + C_tp)

print("Cataract Category: ")
print("-->  True Positives: ", C_tp)
print("-->  True Negatives: ", C_tn)
print("-->  False Positives: ", C_fp)
print("-->  False Negatives: ", C_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(C_tpr))
print("-->  True Negative Rate: {0:.4f}".format(C_tnr))
print("-->  False Positive Rate: {0:.4f}".format(C_fpr))
print("-->  False Negative Rate: {0:.4f}".format(C_fnr))

print()
print('=*=*'*8)
print()

# Glaucoma Evaluation
G_tp = r50_conf_mtrx[2][2]
G_tn = (r50_conf_mtrx[0][0] + r50_conf_mtrx[0][1] + r50_conf_mtrx[0][3] + \
        r50_conf_mtrx[1][0] + r50_conf_mtrx[1][1] + r50_conf_mtrx[1][3] + \
        r50_conf_mtrx[3][0] + r50_conf_mtrx[3][1] + r50_conf_mtrx[3][3])
G_fp = (r50_conf_mtrx[2][0] + r50_conf_mtrx[2][1] + r50_conf_mtrx[2][3])
G_fn = (r50_conf_mtrx[0][2] + r50_conf_mtrx[1][2] + r50_conf_mtrx[3][2])

G_tpr = G_tp / (G_tp + G_fn)
G_tnr = G_tn / (G_tn + G_fp)
G_fpr = G_fp / (G_fp + G_tn)
G_fnr = G_fn / (G_fn + G_tp)

print("Glaucoma Category: ")
print("-->  True Positives: ", G_tp)
print("-->  True Negatives: ", G_tn)
print("-->  False Positives: ", G_fp)
print("-->  False Negatives: ", G_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(G_tpr))
print("-->  True Negative Rate: {0:.4f}".format(G_tnr))
print("-->  False Positive Rate: {0:.4f}".format(G_fpr))
print("-->  False Negative Rate: {0:.4f}".format(G_fnr))

print()
print('=*=*'*8)
print()

# Myopia Evaluation
M_tp = r50_conf_mtrx[3][3]
M_tn = (r50_conf_mtrx[0][0] + r50_conf_mtrx[0][1] + r50_conf_mtrx[0][2] + \
        r50_conf_mtrx[1][0] + r50_conf_mtrx[1][1] + r50_conf_mtrx[1][2] + \
        r50_conf_mtrx[2][0] + r50_conf_mtrx[2][1] + r50_conf_mtrx[2][2])
M_fp = (r50_conf_mtrx[3][0] + r50_conf_mtrx[3][1] + r50_conf_mtrx[3][2])
M_fn = (r50_conf_mtrx[0][3] + r50_conf_mtrx[1][3] + r50_conf_mtrx[2][3])

M_tpr = M_tp / (M_tp + M_fn)
M_tnr = M_tn / (M_tn + M_fp)
M_fpr = M_fp / (M_fp + M_tn)
M_fnr = M_fn / (M_fn + M_tp)

print("Myopia Category: ")
print("-->  True Positives: ", M_tp)
print("-->  True Negatives: ", M_tn)
print("-->  False Positives: ", M_fp)
print("-->  False Negatives: ", M_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(M_tpr))
print("-->  True Negative Rate: {0:.4f}".format(M_tnr))
print("-->  False Positive Rate: {0:.4f}".format(M_fpr))
print("-->  False Negative Rate: {0:.4f}".format(M_fnr))


In [127]:
def confusion_metrics(cm):
#     TP = cm[1][0]
#     TN = cm[0][1]
#     FP = cm[0][0]
#     FN = cm[1][1]

    TP = (N_tp + C_tp + G_tp + M_tp)
    TN = (N_tn + C_tn + G_tn + M_tn)
    FP = (N_fp + C_fp + G_fp + M_fp)
    FN = (N_fn + C_fn + G_fn + M_fn)
    
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
    
    print("True Positives: ", TP)
    print("True Negatives: ", TN)
    print("False Positives: ", FP)
    print("False Negatives: ", FN)
    print('-'*35)
    print("True Positive Rate: {0:.4f}".format(TPR))
    print("True Negative Rate: {0:.4f}".format(TNR))
    print("False Positive Rate: {0:.4f}".format(FPR))
    print("False Negative Rate: {0:.4f}".format(FNR))
    
    # calculate accuracy
    conf_accuracy = (float(TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1 - conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    
    # calculate specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))

    # calculate f1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    
    print('-'*50)
    print(f"Accuracy: {round(conf_accuracy, 4)}")
    print(f"Mis-Classification: {round(conf_misclassification, 4)}")
    print(f"Sensitivity: {round(conf_sensitivity, 4)}")
    print(f"Specificity: {round(conf_specificity,4)}")
    print(f"G-Mean: {round(np.sqrt(round(conf_sensitivity,4) * round(conf_specificity, 4)), 4)}")
    print(f"Precision: {round(conf_precision, 4)}")
    print(f"F1 Score: {round(conf_f1, 4)}")
    
print("ResNet50V2 Confusion Metrics")
confusion_metrics(r50_conf_mtrx)

In [128]:
print("-----InceptionV3 Class Evaluation-----")
# Normal Evaluation
N_tp = inc_conf_mtrx[0][0]
N_fn = (inc_conf_mtrx[1][0] + inc_conf_mtrx[2][0] + inc_conf_mtrx[3][0])
N_tn = (inc_conf_mtrx[1][1] + inc_conf_mtrx[1][2] + inc_conf_mtrx[1][3] + \
        inc_conf_mtrx[2][1] + inc_conf_mtrx[2][2] + inc_conf_mtrx[2][3] + \
        inc_conf_mtrx[3][1] + inc_conf_mtrx[3][2] + inc_conf_mtrx[3][3])
N_fp = (inc_conf_mtrx[0][1] + inc_conf_mtrx[0][2] + inc_conf_mtrx[0][3])

N_tpr = N_tp / (N_tp + N_fn)
N_tnr = N_tn / (N_tn + N_fp)
N_fpr = N_fp / (N_fp + N_tn)
N_fnr = N_fn / (N_fn + N_tp)

print("Normal Category: ")
print("-->  True Positives: ", N_tp)
print("-->  True Negatives: ", N_tn)
print("-->  False Positives: ", N_fp)
print("-->  False Negatives: ", N_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(N_tpr))
print("-->  True Negative Rate: {0:.4f}".format(N_tnr))
print("-->  False Positive Rate: {0:.4f}".format(N_fpr))
print("-->  False Negative Rate: {0:.4f}".format(N_fnr))

print()
print('=*=*'*8)
print()

# Cataract Evaluation
C_tp = inc_conf_mtrx[1][1]
C_tn = (inc_conf_mtrx[0][0] + inc_conf_mtrx[0][2] + inc_conf_mtrx[0][3] + \
        inc_conf_mtrx[2][0] + inc_conf_mtrx[2][2] + inc_conf_mtrx[2][3] + \
        inc_conf_mtrx[3][0] + inc_conf_mtrx[3][2] + inc_conf_mtrx[3][3])
C_fp = (inc_conf_mtrx[1][0] + inc_conf_mtrx[1][2] + inc_conf_mtrx[1][3])
C_fn = (inc_conf_mtrx[0][1] + inc_conf_mtrx[2][1] + inc_conf_mtrx[3][1])

C_tpr = C_tp / (C_tp + C_fn)
C_tnr = C_tn / (C_tn + C_fp)
C_fpr = C_fp / (C_fp + C_tn)
C_fnr = C_fn / (C_fn + C_tp)

print("Cataract Category: ")
print("-->  True Positives: ", C_tp)
print("-->  True Negatives: ", C_tn)
print("-->  False Positives: ", C_fp)
print("-->  False Negatives: ", C_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(C_tpr))
print("-->  True Negative Rate: {0:.4f}".format(C_tnr))
print("-->  False Positive Rate: {0:.4f}".format(C_fpr))
print("-->  False Negative Rate: {0:.4f}".format(C_fnr))

print()
print('=*=*'*8)
print()

# Glaucoma Evaluation
G_tp = inc_conf_mtrx[2][2]
G_tn = (inc_conf_mtrx[0][0] + inc_conf_mtrx[0][1] + inc_conf_mtrx[0][3] + \
        inc_conf_mtrx[1][0] + inc_conf_mtrx[1][1] + inc_conf_mtrx[1][3] + \
        inc_conf_mtrx[3][0] + inc_conf_mtrx[3][1] + inc_conf_mtrx[3][3])
G_fp = (inc_conf_mtrx[2][0] + inc_conf_mtrx[2][1] + inc_conf_mtrx[2][3])
G_fn = (inc_conf_mtrx[0][2] + inc_conf_mtrx[1][2] + inc_conf_mtrx[3][2])

G_tpr = G_tp / (G_tp + G_fn)
G_tnr = G_tn / (G_tn + G_fp)
G_fpr = G_fp / (G_fp + G_tn)
G_fnr = G_fn / (G_fn + G_tp)

print("Glaucoma Category: ")
print("-->  True Positives: ", G_tp)
print("-->  True Negatives: ", G_tn)
print("-->  False Positives: ", G_fp)
print("-->  False Negatives: ", G_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(G_tpr))
print("-->  True Negative Rate: {0:.4f}".format(G_tnr))
print("-->  False Positive Rate: {0:.4f}".format(G_fpr))
print("-->  False Negative Rate: {0:.4f}".format(G_fnr))

print()
print('=*=*'*8)
print()

# Myopia Evaluation
M_tp = inc_conf_mtrx[3][3]
M_tn = (inc_conf_mtrx[0][0] + inc_conf_mtrx[0][1] + inc_conf_mtrx[0][2] + \
        inc_conf_mtrx[1][0] + inc_conf_mtrx[1][1] + inc_conf_mtrx[1][2] + \
        inc_conf_mtrx[2][0] + inc_conf_mtrx[2][1] + inc_conf_mtrx[2][2])
M_fp = (inc_conf_mtrx[3][0] + inc_conf_mtrx[3][1] + inc_conf_mtrx[3][2])
M_fn = (inc_conf_mtrx[0][3] + inc_conf_mtrx[1][3] + inc_conf_mtrx[2][3])

M_tpr = M_tp / (M_tp + M_fn)
M_tnr = M_tn / (M_tn + M_fp)
M_fpr = M_fp / (M_fp + M_tn)
M_fnr = M_fn / (M_fn + M_tp)

print("Myopia Category: ")
print("-->  True Positives: ", M_tp)
print("-->  True Negatives: ", M_tn)
print("-->  False Positives: ", M_fp)
print("-->  False Negatives: ", M_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(M_tpr))
print("-->  True Negative Rate: {0:.4f}".format(M_tnr))
print("-->  False Positive Rate: {0:.4f}".format(M_fpr))
print("-->  False Negative Rate: {0:.4f}".format(M_fnr))


In [138]:
def confusion_metrics(cm):
#     TP = cm[1][0]
#     TN = cm[0][1]
#     FP = cm[0][0]
#     FN = cm[1][1]

    TP = (N_tp + C_tp + G_tp + M_tp)
    TN = (N_tn + C_tn + G_tn + M_tn)
    FP = (N_fp + C_fp + G_fp + M_fp)
    FN = (N_fn + C_fn + G_fn + M_fn)
    
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
    
    print("True Positives: ", TP)
    print("True Negatives: ", TN)
    print("False Positives: ", FP)
    print("False Negatives: ", FN)
    print('-'*35)
    print("True Positive Rate: {0:.4f}".format(TPR))
    print("True Negative Rate: {0:.4f}".format(TNR))
    print("False Positive Rate: {0:.4f}".format(FPR))
    print("False Negative Rate: {0:.4f}".format(FNR))
    
    # calculate accuracy
    conf_accuracy = (float(TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1 - conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    
    # calculate specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))

    # calculate f1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    
    print('-'*50)
    print(f"Accuracy: {round(conf_accuracy, 4)}")
    print(f"Mis-Classification: {round(conf_misclassification, 4)}")
    print(f"Sensitivity: {round(conf_sensitivity, 4)}")
    print(f"Specificity: {round(conf_specificity,4)}")
    print(f"G-Mean: {round(np.sqrt(round(conf_sensitivity,4) * round(conf_specificity, 4)), 4)}")
    print(f"Precision: {round(conf_precision, 4)}")
    print(f"F1 Score: {round(conf_f1, 4)}")
    
print("InceptionV3 Confusion Metrics")
confusion_metrics(inc_conf_mtrx)

In [132]:
print("-----MobileNetV2 Class Evaluation-----")
# Normal Evaluation
N_tp = mobile_conf_mtrx[0][0]
N_fn = (mobile_conf_mtrx[1][0] + mobile_conf_mtrx[2][0] + mobile_conf_mtrx[3][0])
N_tn = (mobile_conf_mtrx[1][1] + mobile_conf_mtrx[1][2] + mobile_conf_mtrx[1][3] + \
        mobile_conf_mtrx[2][1] + mobile_conf_mtrx[2][2] + mobile_conf_mtrx[2][3] + \
        mobile_conf_mtrx[3][1] + mobile_conf_mtrx[3][2] + mobile_conf_mtrx[3][3])
N_fp = (mobile_conf_mtrx[0][1] + mobile_conf_mtrx[0][2] + mobile_conf_mtrx[0][3])

N_tpr = N_tp / (N_tp + N_fn)
N_tnr = N_tn / (N_tn + N_fp)
N_fpr = N_fp / (N_fp + N_tn)
N_fnr = N_fn / (N_fn + N_tp)

print("Normal Category: ")
print("-->  True Positives: ", N_tp)
print("-->  True Negatives: ", N_tn)
print("-->  False Positives: ", N_fp)
print("-->  False Negatives: ", N_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(N_tpr))
print("-->  True Negative Rate: {0:.4f}".format(N_tnr))
print("-->  False Positive Rate: {0:.4f}".format(N_fpr))
print("-->  False Negative Rate: {0:.4f}".format(N_fnr))

print()
print('=*=*'*8)
print()

# Cataract Evaluation
C_tp = mobile_conf_mtrx[1][1]
C_tn = (mobile_conf_mtrx[0][0] + mobile_conf_mtrx[0][2] + mobile_conf_mtrx[0][3] + \
        mobile_conf_mtrx[2][0] + mobile_conf_mtrx[2][2] + mobile_conf_mtrx[2][3] + \
        mobile_conf_mtrx[3][0] + mobile_conf_mtrx[3][2] + mobile_conf_mtrx[3][3])
C_fp = (mobile_conf_mtrx[1][0] + mobile_conf_mtrx[1][2] + mobile_conf_mtrx[1][3])
C_fn = (mobile_conf_mtrx[0][1] + mobile_conf_mtrx[2][1] + mobile_conf_mtrx[3][1])

C_tpr = C_tp / (C_tp + C_fn)
C_tnr = C_tn / (C_tn + C_fp)
C_fpr = C_fp / (C_fp + C_tn)
C_fnr = C_fn / (C_fn + C_tp)

print("Cataract Category: ")
print("-->  True Positives: ", C_tp)
print("-->  True Negatives: ", C_tn)
print("-->  False Positives: ", C_fp)
print("-->  False Negatives: ", C_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(C_tpr))
print("-->  True Negative Rate: {0:.4f}".format(C_tnr))
print("-->  False Positive Rate: {0:.4f}".format(C_fpr))
print("-->  False Negative Rate: {0:.4f}".format(C_fnr))

print()
print('=*=*'*8)
print()

# Glaucoma Evaluation
G_tp = mobile_conf_mtrx[2][2]
G_tn = (mobile_conf_mtrx[0][0] + mobile_conf_mtrx[0][1] + mobile_conf_mtrx[0][3] + \
        mobile_conf_mtrx[1][0] + mobile_conf_mtrx[1][1] + mobile_conf_mtrx[1][3] + \
        mobile_conf_mtrx[3][0] + mobile_conf_mtrx[3][1] + mobile_conf_mtrx[3][3])
G_fp = (mobile_conf_mtrx[2][0] + mobile_conf_mtrx[2][1] + mobile_conf_mtrx[2][3])
G_fn = (mobile_conf_mtrx[0][2] + mobile_conf_mtrx[1][2] + mobile_conf_mtrx[3][2])

G_tpr = G_tp / (G_tp + G_fn)
G_tnr = G_tn / (G_tn + G_fp)
G_fpr = G_fp / (G_fp + G_tn)
G_fnr = G_fn / (G_fn + G_tp)

print("Glaucoma Category: ")
print("-->  True Positives: ", G_tp)
print("-->  True Negatives: ", G_tn)
print("-->  False Positives: ", G_fp)
print("-->  False Negatives: ", G_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(G_tpr))
print("-->  True Negative Rate: {0:.4f}".format(G_tnr))
print("-->  False Positive Rate: {0:.4f}".format(G_fpr))
print("-->  False Negative Rate: {0:.4f}".format(G_fnr))

print()
print('=*=*'*8)
print()

# Myopia Evaluation
M_tp = mobile_conf_mtrx[3][3]
M_tn = (mobile_conf_mtrx[0][0] + mobile_conf_mtrx[0][1] + mobile_conf_mtrx[0][2] + \
        mobile_conf_mtrx[1][0] + mobile_conf_mtrx[1][1] + mobile_conf_mtrx[1][2] + \
        mobile_conf_mtrx[2][0] + mobile_conf_mtrx[2][1] + mobile_conf_mtrx[2][2])
M_fp = (mobile_conf_mtrx[3][0] + mobile_conf_mtrx[3][1] + mobile_conf_mtrx[3][2])
M_fn = (mobile_conf_mtrx[0][3] + mobile_conf_mtrx[1][3] + mobile_conf_mtrx[2][3])

M_tpr = M_tp / (M_tp + M_fn)
M_tnr = M_tn / (M_tn + M_fp)
M_fpr = M_fp / (M_fp + M_tn)
M_fnr = M_fn / (M_fn + M_tp)

print("Myopia Category: ")
print("-->  True Positives: ", M_tp)
print("-->  True Negatives: ", M_tn)
print("-->  False Positives: ", M_fp)
print("-->  False Negatives: ", M_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(M_tpr))
print("-->  True Negative Rate: {0:.4f}".format(M_tnr))
print("-->  False Positive Rate: {0:.4f}".format(M_fpr))
print("-->  False Negative Rate: {0:.4f}".format(M_fnr))


In [131]:
def confusion_metrics(cm):
#     TP = cm[1][0]
#     TN = cm[0][1]
#     FP = cm[0][0]
#     FN = cm[1][1]

    TP = (N_tp + C_tp + G_tp + M_tp)
    TN = (N_tn + C_tn + G_tn + M_tn)
    FP = (N_fp + C_fp + G_fp + M_fp)
    FN = (N_fn + C_fn + G_fn + M_fn)
    
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
    
    print("True Positives: ", TP)
    print("True Negatives: ", TN)
    print("False Positives: ", FP)
    print("False Negatives: ", FN)
    print('-'*35)
    print("True Positive Rate: {0:.4f}".format(TPR))
    print("True Negative Rate: {0:.4f}".format(TNR))
    print("False Positive Rate: {0:.4f}".format(FPR))
    print("False Negative Rate: {0:.4f}".format(FNR))
    
    # calculate accuracy
    conf_accuracy = (float(TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1 - conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    
    # calculate specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))

    # calculate f1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    
    print('-'*50)
    print(f"Accuracy: {round(conf_accuracy, 4)}")
    print(f"Mis-Classification: {round(conf_misclassification, 4)}")
    print(f"Sensitivity: {round(conf_sensitivity, 4)}")
    print(f"Specificity: {round(conf_specificity,4)}")
    print(f"G-Mean: {round(np.sqrt(round(conf_sensitivity,4) * round(conf_specificity, 4)), 4)}")
    print(f"Precision: {round(conf_precision, 4)}")
    print(f"F1 Score: {round(conf_f1, 4)}")
    
print("MobileNetV2 Confusion Metrics")
confusion_metrics(mobile_conf_mtrx)

In [141]:
print("-----DenseNet169 Class Evaluation-----")
# Normal Evaluation
N_tp = dense_conf_mtrx[0][0]
N_fn = (dense_conf_mtrx[1][0] + dense_conf_mtrx[2][0] + dense_conf_mtrx[3][0])
N_tn = (dense_conf_mtrx[1][1] + dense_conf_mtrx[1][2] + dense_conf_mtrx[1][3] + \
        dense_conf_mtrx[2][1] + dense_conf_mtrx[2][2] + dense_conf_mtrx[2][3] + \
        dense_conf_mtrx[3][1] + dense_conf_mtrx[3][2] + dense_conf_mtrx[3][3])
N_fp = (dense_conf_mtrx[0][1] + dense_conf_mtrx[0][2] + dense_conf_mtrx[0][3])

N_tpr = N_tp / (N_tp + N_fn)
N_tnr = N_tn / (N_tn + N_fp)
N_fpr = N_fp / (N_fp + N_tn)
N_fnr = N_fn / (N_fn + N_tp)

print("Normal Category: ")
print("-->  True Positives: ", N_tp)
print("-->  True Negatives: ", N_tn)
print("-->  False Positives: ", N_fp)
print("-->  False Negatives: ", N_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(N_tpr))
print("-->  True Negative Rate: {0:.4f}".format(N_tnr))
print("-->  False Positive Rate: {0:.4f}".format(N_fpr))
print("-->  False Negative Rate: {0:.4f}".format(N_fnr))

print()
print('=*=*'*8)
print()

# Cataract Evaluation
C_tp = dense_conf_mtrx[1][1]
C_tn = (dense_conf_mtrx[0][0] + dense_conf_mtrx[0][2] + dense_conf_mtrx[0][3] + \
        dense_conf_mtrx[2][0] + dense_conf_mtrx[2][2] + dense_conf_mtrx[2][3] + \
        dense_conf_mtrx[3][0] + dense_conf_mtrx[3][2] + dense_conf_mtrx[3][3])
C_fp = (dense_conf_mtrx[1][0] + dense_conf_mtrx[1][2] + dense_conf_mtrx[1][3])
C_fn = (dense_conf_mtrx[0][1] + dense_conf_mtrx[2][1] + dense_conf_mtrx[3][1])

C_tpr = C_tp / (C_tp + C_fn)
C_tnr = C_tn / (C_tn + C_fp)
C_fpr = C_fp / (C_fp + C_tn)
C_fnr = C_fn / (C_fn + C_tp)

print("Cataract Category: ")
print("-->  True Positives: ", C_tp)
print("-->  True Negatives: ", C_tn)
print("-->  False Positives: ", C_fp)
print("-->  False Negatives: ", C_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(C_tpr))
print("-->  True Negative Rate: {0:.4f}".format(C_tnr))
print("-->  False Positive Rate: {0:.4f}".format(C_fpr))
print("-->  False Negative Rate: {0:.4f}".format(C_fnr))

print()
print('=*=*'*8)
print()

# Glaucoma Evaluation
G_tp = dense_conf_mtrx[2][2]
G_tn = (dense_conf_mtrx[0][0] + dense_conf_mtrx[0][1] + dense_conf_mtrx[0][3] + \
        dense_conf_mtrx[1][0] + dense_conf_mtrx[1][1] + dense_conf_mtrx[1][3] + \
        dense_conf_mtrx[3][0] + dense_conf_mtrx[3][1] + dense_conf_mtrx[3][3])
G_fp = (dense_conf_mtrx[2][0] + dense_conf_mtrx[2][1] + dense_conf_mtrx[2][3])
G_fn = (dense_conf_mtrx[0][2] + dense_conf_mtrx[1][2] + dense_conf_mtrx[3][2])

G_tpr = G_tp / (G_tp + G_fn)
G_tnr = G_tn / (G_tn + G_fp)
G_fpr = G_fp / (G_fp + G_tn)
G_fnr = G_fn / (G_fn + G_tp)

print("Glaucoma Category: ")
print("-->  True Positives: ", G_tp)
print("-->  True Negatives: ", G_tn)
print("-->  False Positives: ", G_fp)
print("-->  False Negatives: ", G_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(G_tpr))
print("-->  True Negative Rate: {0:.4f}".format(G_tnr))
print("-->  False Positive Rate: {0:.4f}".format(G_fpr))
print("-->  False Negative Rate: {0:.4f}".format(G_fnr))

print()
print('=*=*'*8)
print()

# Myopia Evaluation
M_tp = dense_conf_mtrx[3][3]
M_tn = (dense_conf_mtrx[0][0] + dense_conf_mtrx[0][1] + dense_conf_mtrx[0][2] + \
        dense_conf_mtrx[1][0] + dense_conf_mtrx[1][1] + dense_conf_mtrx[1][2] + \
        dense_conf_mtrx[2][0] + dense_conf_mtrx[2][1] + dense_conf_mtrx[2][2])
M_fp = (dense_conf_mtrx[3][0] + dense_conf_mtrx[3][1] + dense_conf_mtrx[3][2])
M_fn = (dense_conf_mtrx[0][3] + dense_conf_mtrx[1][3] + dense_conf_mtrx[2][3])

M_tpr = M_tp / (M_tp + M_fn)
M_tnr = M_tn / (M_tn + M_fp)
M_fpr = M_fp / (M_fp + M_tn)
M_fnr = M_fn / (M_fn + M_tp)

print("Myopia Category: ")
print("-->  True Positives: ", M_tp)
print("-->  True Negatives: ", M_tn)
print("-->  False Positives: ", M_fp)
print("-->  False Negatives: ", M_fn)
print('-'*32)
print("-->  True Positive Rate: {0:.4f}".format(M_tpr))
print("-->  True Negative Rate: {0:.4f}".format(M_tnr))
print("-->  False Positive Rate: {0:.4f}".format(M_fpr))
print("-->  False Negative Rate: {0:.4f}".format(M_fnr))


In [142]:
def confusion_metrics(cm):
#     TP = cm[1][0]
#     TN = cm[0][1]
#     FP = cm[0][0]
#     FN = cm[1][1]

    TP = (N_tp + C_tp + G_tp + M_tp)
    TN = (N_tn + C_tn + G_tn + M_tn)
    FP = (N_fp + C_fp + G_fp + M_fp)
    FN = (N_fn + C_fn + G_fn + M_fn)
    
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
    
    print("True Positives: ", TP)
    print("True Negatives: ", TN)
    print("False Positives: ", FP)
    print("False Negatives: ", FN)
    print('-'*35)
    print("True Positive Rate: {0:.4f}".format(TPR))
    print("True Negative Rate: {0:.4f}".format(TNR))
    print("False Positive Rate: {0:.4f}".format(FPR))
    print("False Negative Rate: {0:.4f}".format(FNR))
    
    # calculate accuracy
    conf_accuracy = (float(TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1 - conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    
    # calculate specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))

    # calculate f1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    
    print('-'*50)
    print(f"Accuracy: {round(conf_accuracy, 4)}")
    print(f"Mis-Classification: {round(conf_misclassification, 4)}")
    print(f"Sensitivity: {round(conf_sensitivity, 4)}")
    print(f"Specificity: {round(conf_specificity,4)}")
    print(f"G-Mean: {round(np.sqrt(round(conf_sensitivity,4) * round(conf_specificity, 4)), 4)}")
    print(f"Precision: {round(conf_precision, 4)}")
    print(f"F1 Score: {round(conf_f1, 4)}")
    
print("DenseNet169 Confusion Metrics")
confusion_metrics(dense_cm)

**Kappa Score**

In [153]:
from sklearn import metrics
from matplotlib.cbook import flatten

th = 0.5
# gt = (test_generator.classes)
# gt = (np.argmax(y_test, axis=1))
rc_y_pred_raw = resnet_crelu_model.predict(x_test1, verbose=1)
r50_y_pred_raw = resnet50_v2_model.predict(x_test1, verbose=1)
inc_y_pred_raw = inceptionv3_model.predict(x_test1, verbose=1)
mobile_y_pred_raw = mobilenetv2_model.predict(x_test1, verbose=1)
dense_y_pred_raw = densenet169_model.predict(x_test1, verbose=1)

gt = np.argmax(y_test, axis=1)
rc_pr = np.argmax(rc_y_pred_raw, axis=1)
r50_pr = np.argmax(r50_y_pred_raw, axis=1)
inc_pr = np.argmax(inc_y_pred_raw, axis=1)
mobile_pr = np.argmax(mobile_y_pred_raw, axis=1)
dense_pr = np.argmax(dense_y_pred_raw, axis=1)

rc_kappa = metrics.cohen_kappa_score(gt,rc_pr)
r50_kappa = metrics.cohen_kappa_score(gt,r50_pr)
inc_kappa = metrics.cohen_kappa_score(gt,inc_pr)
mobile_kappa = metrics.cohen_kappa_score(gt,mobile_pr)
dense_kappa = metrics.cohen_kappa_score(gt,dense_pr)

# auc = metrics.roc_auc_score(gt, pr, multi_class='ovr')
rc_f1 = metrics.f1_score(gt, rc_pr, average='micro')
r50_f1 = metrics.f1_score(gt, r50_pr, average='micro')
inc_f1 = metrics.f1_score(gt, inc_pr, average='micro')
mobile_f1 = metrics.f1_score(gt, mobile_pr, average='micro')
dense_f1 = metrics.f1_score(gt, dense_pr, average='micro')


# final_score = (kappa + f1 + auc) / 3.0

print("Evaluation Metrics: ")
print('RC')
print("-> Kappa Score: {0:.4f}".format(rc_kappa))
print("-> F1 Score: {0:.4f}".format(rc_f1))
print('R50')
print("-> Kappa Score: {0:.4f}".format(r50_kappa))
print("-> F1 Score: {0:.4f}".format(r50_f1))
print('Inception')
print("-> Kappa Score: {0:.4f}".format(inc_kappa))
print("-> F1 Score: {0:.4f}".format(inc_f1))
print('Mobile')
print("-> Kappa Score: {0:.4f}".format(mobile_kappa))
print("-> F1 Score: {0:.4f}".format(mobile_f1))
print('Dense')
print("-> Kappa Score: {0:.4f}".format(dense_kappa))
print("-> F1 Score: {0:.4f}".format(dense_f1))
# print("-> AUC Score: {0:.4f}".format(auc))
# print("-> Final Score: {0:.4f}".format(final_score))

